<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/03-Loss%E4%B8%8E%E4%BC%98%E5%8C%96/DeterministicNeuralSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

## 松弛 NerualSort
- batch_size = 4
- seq_size = 3

In [2]:
def deterministic_NeuralSort(inputs, N, tau):
    batch_size = tf.shape(inputs)[0]
    one_mat = tf.ones([batch_size, N, N])

    scaling = tf.cast(N + 1 - 2 * (tf.range(N) + 1), dtype = tf.float32)
    scaling = tf.tile(tf.reshape(scaling, [1, 1, -1]), [batch_size, 1, 1])

    A = tf.abs(inputs - tf.transpose(inputs, perm=[0, 2, 1]))
    B = tf.matmul(A, one_mat)
    C = tf.matmul(inputs, scaling)

    p_logits = tf.transpose(C - B, perm=[0, 2, 1])
    p_logistic = tf.nn.softmax(p_logits / tau, -1)
    return p_logistic

In [3]:
batch_size = 4
N = 3
one_mat = tf.ones([batch_size, N, N])
one_mat

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [4]:
inputs = tf.random.uniform([batch_size, N, 1])
A = tf.abs(inputs - tf.transpose(inputs, perm=[0, 2, 1]))
B = tf.matmul(A, one_mat)
inputs, A, B

(<tf.Tensor: shape=(4, 3, 1), dtype=float32, numpy=
 array([[[0.8223963 ],
         [0.74837196],
         [0.94974554]],
 
        [[0.11972165],
         [0.7227521 ],
         [0.42933714]],
 
        [[0.8004824 ],
         [0.16805673],
         [0.3474642 ]],
 
        [[0.62646854],
         [0.94854116],
         [0.4159677 ]]], dtype=float32)>,
 <tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
 array([[[0.        , 0.07402432, 0.12734926],
         [0.07402432, 0.        , 0.20137358],
         [0.12734926, 0.20137358, 0.        ]],
 
        [[0.        , 0.60303044, 0.3096155 ],
         [0.60303044, 0.        , 0.29341495],
         [0.3096155 , 0.29341495, 0.        ]],
 
        [[0.        , 0.63242567, 0.4530182 ],
         [0.63242567, 0.        , 0.17940748],
         [0.4530182 , 0.17940748, 0.        ]],
 
        [[0.        , 0.32207263, 0.21050084],
         [0.32207263, 0.        , 0.53257346],
         [0.21050084, 0.53257346, 0.        ]]], dtype=float32)>,


In [5]:
scaling = tf.cast(N + 1 - 2 * (tf.range(N) + 1), dtype = tf.float32)
scaling

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 2.,  0., -2.], dtype=float32)>

In [6]:
scaling = tf.tile(tf.reshape(scaling, [1, 1, -1]), [batch_size, 1, 1])
scaling

<tf.Tensor: shape=(4, 1, 3), dtype=float32, numpy=
array([[[ 2.,  0., -2.]],

       [[ 2.,  0., -2.]],

       [[ 2.,  0., -2.]],

       [[ 2.,  0., -2.]]], dtype=float32)>

In [7]:
C = tf.matmul(inputs, scaling) # 4,3,1 x 4,1,3
C

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[ 1.6447926 ,  0.        , -1.6447926 ],
        [ 1.4967439 ,  0.        , -1.4967439 ],
        [ 1.8994911 ,  0.        , -1.8994911 ]],

       [[ 0.2394433 ,  0.        , -0.2394433 ],
        [ 1.4455042 ,  0.        , -1.4455042 ],
        [ 0.8586743 ,  0.        , -0.8586743 ]],

       [[ 1.6009648 ,  0.        , -1.6009648 ],
        [ 0.33611345,  0.        , -0.33611345],
        [ 0.6949284 ,  0.        , -0.6949284 ]],

       [[ 1.2529371 ,  0.        , -1.2529371 ],
        [ 1.8970823 ,  0.        , -1.8970823 ],
        [ 0.8319354 ,  0.        , -0.8319354 ]]], dtype=float32)>

In [8]:
p_logits = tf.transpose(C - B, perm=[0, 2, 1])
p_logits

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[ 1.443419  ,  1.221346  ,  1.5707682 ],
        [-0.20137358, -0.2753979 , -0.32872283],
        [-1.8461661 , -1.7721418 , -2.2282138 ]],

       [[-0.67320263,  0.5490588 ,  0.25564384],
        [-0.91264594, -0.8964454 , -0.60303044],
        [-1.1520892 , -2.3419495 , -1.4617047 ]],

       [[ 0.51552093, -0.4757197 ,  0.06250274],
        [-1.0854439 , -0.81183314, -0.63242567],
        [-2.6864085 , -1.1479466 , -1.3273541 ]],

       [[ 0.7203636 ,  1.0424362 ,  0.08886111],
        [-0.53257346, -0.8546461 , -0.7430743 ],
        [-1.7855105 , -2.7517285 , -1.5750097 ]]], dtype=float32)>

In [9]:
p_logistic = tf.nn.softmax(p_logits, -1)
p_logistic

<tf.Tensor: shape=(4, 3, 3), dtype=float32, numpy=
array([[[0.3405217 , 0.27270916, 0.38676918],
        [0.35598904, 0.3305889 , 0.31342205],
        [0.36241132, 0.39025646, 0.24733214]],

       [[0.14437425, 0.4901298 , 0.36549598],
        [0.2959251 , 0.3007583 , 0.40331662],
        [0.4906788 , 0.14929579, 0.36002526]],

       [[0.49830014, 0.18492715, 0.31677267],
        [0.25721782, 0.33816504, 0.40461722],
        [0.10471278, 0.48769155, 0.40759566]],

       [[0.3434329 , 0.47393224, 0.18263492],
        [0.39450473, 0.2858761 , 0.3196192 ],
        [0.38243616, 0.1455244 , 0.47203943]]], dtype=float32)>

## 模拟 listwise 6选3

In [10]:
N = 6
TopK = 3

In [11]:
rank_score = tf.random.uniform([4, N], minval=0.0, maxval=1.0)
rank_score

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.19664598, 0.2104162 , 0.10328329, 0.25442326, 0.04811656,
        0.45724213],
       [0.9729967 , 0.3479097 , 0.63081527, 0.2376535 , 0.76290333,
        0.3004533 ],
       [0.6977751 , 0.66400313, 0.89002   , 0.37777936, 0.1722467 ,
        0.15054572],
       [0.00286019, 0.88245285, 0.29435456, 0.4042896 , 0.84069896,
        0.735322  ]], dtype=float32)>

In [12]:
# 每行样本的topK下标
groundtruth = tf.nn.top_k(rank_score, TopK)[1]
groundtruth

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[5, 3, 1],
       [0, 4, 2],
       [2, 0, 1],
       [1, 4, 5]], dtype=int32)>

In [13]:
target_label_matrix = tf.one_hot(groundtruth, N)
target_sort_matrix = tf.reshape(target_label_matrix, [-1, N])
target_sort_matrix

<tf.Tensor: shape=(12, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [14]:
listwise_score = tf.random.uniform([4, N, 1])
listwise_score

<tf.Tensor: shape=(4, 6, 1), dtype=float32, numpy=
array([[[0.660874  ],
        [0.5152323 ],
        [0.5283922 ],
        [0.9928918 ],
        [0.6450194 ],
        [0.8328718 ]],

       [[0.8678943 ],
        [0.4251473 ],
        [0.2908268 ],
        [0.73846936],
        [0.7699381 ],
        [0.0582037 ]],

       [[0.76821876],
        [0.8891057 ],
        [0.21235633],
        [0.94094944],
        [0.13954937],
        [0.0775156 ]],

       [[0.50557864],
        [0.95465577],
        [0.37362206],
        [0.78056335],
        [0.59861195],
        [0.16696286]]], dtype=float32)>

In [15]:
score_sort_matrix = deterministic_NeuralSort(listwise_score, N, 1.0)
score_sort_matrix

<tf.Tensor: shape=(4, 6, 6), dtype=float32, numpy=
array([[[0.1578238 , 0.05724711, 0.06444525, 0.31028548, 0.14579573,
         0.2644026 ],
        [0.19339082, 0.09386843, 0.10292631, 0.19572113, 0.18440774,
         0.22968546],
        [0.21320048, 0.13847604, 0.14789414, 0.11107161, 0.20984693,
         0.17951076],
        [0.21485291, 0.18673703, 0.1942568 , 0.05761946, 0.21828645,
         0.12824728],
        [0.2019629 , 0.2348895 , 0.2380011 , 0.02788127, 0.2118012 ,
         0.08546401],
        [0.18031587, 0.28062648, 0.27695763, 0.01281409, 0.19519185,
         0.05409411]],

       [[0.33563066, 0.05780288, 0.02257392, 0.27689794, 0.3043125 ,
         0.00278205],
        [0.26012453, 0.10860112, 0.05548314, 0.27800745, 0.28689513,
         0.01088861],
        [0.1777463 , 0.17989482, 0.12023052, 0.2460893 , 0.23846576,
         0.03757337],
        [0.0990925 , 0.24312186, 0.21256383, 0.17772558, 0.16171485,
         0.10578125],
        [0.04264425, 0.25363487, 0.29

In [16]:
score_sort_matrix = tf.reshape(score_sort_matrix[:, 0:TopK, :], [-1, N])
score_sort_matrix = tf.clip_by_value(score_sort_matrix, clip_value_min=1e-9, clip_value_max=1.0)
score_sort_matrix

<tf.Tensor: shape=(12, 6), dtype=float32, numpy=
array([[0.1578238 , 0.05724711, 0.06444525, 0.31028548, 0.14579573,
        0.2644026 ],
       [0.19339082, 0.09386843, 0.10292631, 0.19572113, 0.18440774,
        0.22968546],
       [0.21320048, 0.13847604, 0.14789414, 0.11107161, 0.20984693,
        0.17951076],
       [0.33563066, 0.05780288, 0.02257392, 0.27689794, 0.3043125 ,
        0.00278205],
       [0.26012453, 0.10860112, 0.05548314, 0.27800745, 0.28689513,
        0.01088861],
       [0.1777463 , 0.17989482, 0.12023052, 0.2460893 , 0.23846576,
        0.03757337],
       [0.24561088, 0.35297927, 0.01524782, 0.37176168, 0.00915946,
        0.00524085],
       [0.2774125 , 0.3130592 , 0.05234824, 0.2972426 , 0.03637507,
        0.02356228],
       [0.24892165, 0.22057797, 0.14277583, 0.18880598, 0.11476135,
        0.08415716],
       [0.11024622, 0.3606217 , 0.04377314, 0.30300134, 0.17554279,
        0.00681474],
       [0.17010598, 0.22664404, 0.08793859, 0.2697439 , 0.224

In [17]:
target_sort_matrix, score_sort_matrix

(<tf.Tensor: shape=(12, 6), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(12, 6), dtype=float32, numpy=
 array([[0.1578238 , 0.05724711, 0.06444525, 0.31028548, 0.14579573,
         0.2644026 ],
        [0.19339082, 0.09386843, 0.10292631, 0.19572113, 0.18440774,
         0.22968546],
        [0.21320048, 0.13847604, 0.14789414, 0.11107161, 0.20984693,
         0.17951076],
        [0.33563066, 0.05780288, 0.02257392, 0.27689794, 0.3043125 ,
         0.00278205],
        [0.26012453, 0.10860112, 0.05548314, 0.27800745, 0.28689513,
         0.01088861],
        [0.1777463 

In [18]:
topk_loss = -1.0 * tf.reduce_sum(target_sort_matrix * tf.math.log(score_sort_matrix), axis=1)
topk_loss

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([1.3302823, 1.6310644, 1.9770579, 1.091744 , 1.2486385, 2.1183443,
       4.183319 , 1.2822497, 1.511504 , 1.0199258, 1.4922338, 2.9260447],
      dtype=float32)>

In [19]:
listwise_loss = tf.reduce_sum(tf.reshape(topk_loss, [-1, TopK]), axis=1)
listwise_loss

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([4.9384046, 4.458727 , 6.9770727, 5.4382043], dtype=float32)>